## Installation and deployment of 5G RAN (gNB) of Open Air Interface (OAI)
## Objectives
- Install and configure the OAI gNB 
- Deploy a gNB together a previous configured 5GC
- Inspec gNB status in 5GC Core

At the end of a set of three hands-on (this is the second one), the goal is to run the simulated OAI scenario, which consists of the 5G core (5GC), gNB in CU/DU split mode, and user equipment (UE), according to the image below: 

![5GC_01](./FIGS/H01_5GCore_UNI_III/5GC_deploy.jpg)

Now, we are going to install, configure, deploy and interact to gNB with CU/DU split mode. 


## gNB Installation

To install OAI RAN 5G, go back to /home/dco1020 and follow this tutorial.

**Step 0:** go back to /home/dco1020. In a Linux Terminal, type:

sudo su

cd /home/dco1020


**Step 1:** Install dependencies

sudo add-apt-repository ppa:ettusresearch/uhd

sudo apt-get install libuhd-dev uhd-host -y

sudo apt-get install libuhd4.2.0 -y

sudo apt-get install libuhd4.4.0 -y

**Step 2:** Fix problem in the libuhd. When you run the commands below, if you get the error "cannot access archive", you can continue to the next step.

sudo dpkg -i --force-overwrite /var/cache/apt/archives/libuhd4.2.0_4.2.0.1-0ubuntu1~focal1_amd64.deb

sudo dpkg -i --force-overwrite /var/cache/apt/archives/libuhd4.4.0_4.4.0.0-0ubuntu1~focal1_amd64.deb

**Step 3:** Clone OpenAirInterface 5G RAN repository

git clone https://gitlab.eurecom.fr/oai/openairinterface5g.git

cd /home/dco1020/openairinterface5g

git checkout 2023.w47

**Step 4:** Run the installation script

source oaienv

cd cmake_targets

sudo ./build_oai -I 

sudo ./build_oai -I --install-optional-packages

sudo ./build_oai --gNB --eNB --UE --nrUE 

**Attention**: you may find Wireshark configuration screen, select "yes" to continue.

**Attention**: be patient, the last command could take a whileeeee.

If you have any compilation error, please, try:

./build_oai -c -C

./build_oai -I

sudo ./build_oai -I --install-optional-packages 

sudo ./build_oai --gNB --eNB --UE --nrUE

Take care about conflits between our VM and some windows configuration, If you have compilation problem, please, check and disable WSL, hipervison and virtual machine platforms, acoording to the following figures.


![gNB_problem1](./FIGS/H02_5GRAN_UNI_III/gNB_problem1.png)

![gNB_problem2](./FIGS/H02_5GRAN_UNI_III/gNB_problem2.png)

## gNB Configuration

In a Linux Terminal, follow the steps below:

**Step 1:** Configure the PLMN and TAC values

The Public Land Mobile Network (PLMN) and Tracking Area Code (TAC) will be configured in the CU (/home/dco1020/openairinterface5g/targets/PROJECTS/GENERIC-NR-5GC/CONF/cu_gnb.conf) and DU (/home/dco1020/openairinterface5g/targets/PROJECTS/GENERIC-NR-5GC/CONF/du_gnb.conf) configuration files. 

Figure below shows the part of the code that must be modified in the CU and DU files (lines 17 and 18). The parameters shown in figure should be modified according to the values present in docker compose in the "oai-amf" session (/home/dco1020/oai-cn5g-fed/docker-compose/docker-compose-basic-nrf.yaml). 

![gNB_config_01](./FIGS/H02_5GRAN_UNI_III/gNB_config_01.png)


Figure below shows the parameters that should be checked and passed to the CU and DU configuration files: PLMN_SUPPORT_TAC (referring to "tracking_area_code"), PLMN_SUPPORT_MCC (referring to "mcc"), PLMN_SUPPORT_MNC (referring to "mnc"), SST_0 (referring to "sst") and the SD_0 (referring to "sd"). If the "sd" parameter is not in the CU and DU configuration files, simply add it according to line 18 in previous figure.

![gNB_config_02](./FIGS/H02_5GRAN_UNI_III/gNB_config_02.png)


Open files by typing:

gedit /home/dco1020/openairinterface5g/targets/PROJECTS/GENERIC-NR-5GC/CONF/cu_gnb.conf

gedit /home/dco1020/openairinterface5g/targets/PROJECTS/GENERIC-NR-5GC/CONF/du_gnb.conf

gedit /home/dco1020/oai-cn5g-fed/docker-compose/docker-compose-basic-nrf.yaml

**Step 2:**  Configure the network interfaces

This part you will configure the N2 interface in (/home/dco1020/openairinterface5g/targets/PROJECTS/GENERIC-NR-5GC/CONF/cu_gnb.conf), such as in figure below. This interface is responsible to connect the CU and the AMF.

![gNB_config_03](./FIGS/H02_5GRAN_UNI_III/gNB_config_03.png)

You need to verify/change the IP addresses. You can find the OAI-AMF IP address inside /home/dco1020/oai-cn5g-fed/docker-compose/docker-compose-basic-nrf.yaml, such as in figure below. You must get this IP address and put it in the "amf_ip_address" parameter, in the "ipv4" field (such as in previous figure).

![gNB_config_04](./FIGS/H02_5GRAN_UNI_III/gNB_config_04.png)


Check if the IP Address of cu_gnb.conf is the same of docker-compose-basic-nrf.yaml. To edit cu_gnb.conf, type:

gedit /home/dco1020/openairinterface5g/targets/PROJECTS/GENERIC-NR-5GC/CONF/cu_gnb.conf


**Step 3:** to check a virtual interface (demo-oai), type:

ifconfig -a

You should find a virtual interface (demo-oai) (as shown in figure below). This interface serves to connect the CU with OAI-AMF.  You must get its IP address and name to configure the N2 interface such as two figures before. 

![gNB_config_05](./FIGS/H02_5GRAN_UNI_III/gNB_config_05.png)

## Deployment

The deployment of the CU and DU is done separately, and the CU needs to be started before the DU. 

**NOTE:** The CU and DU will be running on two different terminals

**Step 1:** open a new terminal using crtl+Alt+T and type:

sudo su

cd /home/dco1020/openairinterface5g/cmake_targets/ran_build/build/

sudo RFSIMULATOR=server ./nr-softmodem --rfsim --sa -O ../../../targets/PROJECTS/GENERIC-NR-5GC/CONF/cu_gnb.conf --continuous-tx 

**Step 2:** Open another terminal using crtl+Alt+T and type: 

sudo su 

cd /home/dco1020/openairinterface5g/cmake_targets/ran_build/build/

sudo RFSIMULATOR=server ./nr-softmodem --rfsim --sa -O ../../../targets/PROJECTS/GENERIC-NR-5GC/CONF/du_gnb.conf --continuous-tx --gNBs.[0].min_rxtxtime 6


**Step 3**: To check the connection of gNodeB to the Core, see AMF logs. Open another terminal (the third one) using crtl+Alt+T and type: 

docker logs -f oai-amf

The expected return is shown in figure below, where gNodeB shows the connected status.

![gNB_config_06](./FIGS/H02_5GRAN_UNI_III/gNB_config_06.png)

# Deliverable 02:

- Inspec the gNB configuration files and change its name to gNB-DCO1020. Show the new gNB name in the AMF log file.